## fn-given-filepaths
### Description
Endpoint for generating Gromet JSON from a serialized code system.

### Required Arguments
- files list[str] - The relative file path from the directory specified by `root_name`, corresponding to each entry in `blobs`
- blobs list[str] - Contents of each file to be analyzed

### Optional Arguments
- system_name str - A model name to associate with the provided code
- root_name str - The name of the code system's root directory
- comments CodeComments - A CodeComments object representing the comments extracted from the source code in 'blobs'. Can provide comments for a single file (SingleFileCodeComments) or multiple files (MultiFileCodeComments)
- dependency_depth int - The depth at which to ingest dependencies into Gromet. i.e. 0=Ingest no dependencies, 1=Ingest system dependencies. Accepted values: [0,1]

## Imports

In [1]:
import requests 
import os
from pathlib import Path
from typing import Dict

## Generate endpoint URL

In [2]:
SKEMA_ADDRESS = os.environ.get("SKEMA_ADDRESS", "http://localhost:8000")
URL = f"{SKEMA_ADDRESS}/code2fn/fn-given-filepaths"

## Process a single file 

In [3]:
post_data = {
    "files":["exp0.py"],
    "blobs":["x=2"]
}

response_single = requests.post(URL, json=post_data)
print(response_single.json())

{'schema': 'FN', 'schema_version': '0.1.7', 'name': '', 'modules': [{'schema': 'FN', 'schema_version': '0.1.7', 'name': 'exp0', 'fn': {'name': None, 'b': [{'function_type': 'MODULE', 'body': None, 'import_type': None, 'import_version': None, 'import_source': None, 'source_language': None, 'source_language_version': None, 'value': None, 'gromet_type': 'GrometBoxFunction', 'name': 'module', 'metadata': 3}], 'opi': None, 'opo': None, 'wopio': None, 'bf': [{'function_type': 'EXPRESSION', 'body': 1, 'import_type': None, 'import_version': None, 'import_source': None, 'source_language': None, 'source_language_version': None, 'value': None, 'gromet_type': 'GrometBoxFunction', 'name': None, 'metadata': 5}], 'pif': None, 'pof': [{'id': 1, 'name': 'x', 'box': 1, 'default_value': None, 'gromet_type': 'GrometPort', 'metadata': None}], 'wfopi': None, 'wfl': None, 'wff': None, 'wfc': None, 'wfopo': None, 'bl': None, 'pil': None, 'pol': None, 'wlopi': None, 'wll': None, 'wlf': None, 'wlc': None, 'wlop

## Process a system

In [4]:
post_data = {
    "files":["exp0.py", "exp1.py"],
    "blobs":["x=2", "x=2\ny=1"],
}

response_multi = requests.post(URL, json=post_data)
print(response_multi.json())

{'schema': 'FN', 'schema_version': '0.1.7', 'name': 'system_test', 'modules': [{'schema': 'FN', 'schema_version': '0.1.7', 'name': 'driver', 'fn': {'name': None, 'b': [{'function_type': 'MODULE', 'body': None, 'import_type': None, 'import_version': None, 'import_source': None, 'source_language': None, 'source_language_version': None, 'value': None, 'gromet_type': 'GrometBoxFunction', 'name': 'module', 'metadata': 3}], 'opi': None, 'opo': None, 'wopio': None, 'bf': [{'function_type': 'EXPRESSION', 'body': None, 'import_type': None, 'import_version': None, 'import_source': None, 'source_language': None, 'source_language_version': None, 'value': None, 'gromet_type': 'GrometBoxFunction', 'name': 'model', 'metadata': None}], 'pif': None, 'pof': [{'id': 1, 'name': 'model', 'box': 1, 'default_value': None, 'gromet_type': 'GrometPort', 'metadata': None}], 'wfopi': None, 'wfl': None, 'wff': None, 'wfc': None, 'wfopo': None, 'bl': None, 'pil': None, 'pol': None, 'wlopi': None, 'wll': None, 'wlf'

## Process a system with optional arguments

In [ ]:
post_data = {
    "files":["example1.py", "dir/example2.py"],
    "blobs":["import minimal\nx=minimal.add(1,2)", "#Variable declaration\nx=2\n#Function definition\ndef foo(x):\n    '''Increment the input variable'''\n    return x+1",],
    "system_name": "examples",
    "root_name": "example-system",
    "comments": {
            "files": {
                "example-system/dir/example2.py": {
                    "single": [
                        {"content": "Variable declaration", "line_number": 0},
                        {"content": "Function definition", "line_number": 2},
                    ],
                    "multi": [],
                    "docstring": [
                        {
                            "content": ["Increment the input variable"],
                            "function_name": "foo",
                            "start_line_number": 5,
                            "end_line_number": 6,
                        }
                    ],
                }
            }
        },
    "dependency_depth": 1
}

response_multi = requests.post(URL, json=post_data)
print(response_multi.json())